In [ ]:
!pip install python-docx pandas scikit-elm statsmodels

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
import docx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from docx import Document
import numpy as np
import pandas as pd
import time
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from docx import Document
from numpy.linalg import inv

In [ ]:
def read_data_from_docx(file_path):
    doc = docx.Document(file_path)
    data = []
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text for cell in row.cells]
            data.append(row_data)
    return data

def write_data_to_csv(data, file_path):
    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False, header=False)

docx_file_path = 'Bostondata.docx'
csv_file_path = 'Bostondata.csv'
data = read_data_from_docx(docx_file_path)
write_data_to_csv(data, csv_file_path)
df = pd.read_csv(csv_file_path)
print(df.head())

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from docx import Document
import time

class MixedRegressionELM:
    def __init__(self, hidden_units, activation_function='sigmoid', regularization=1e-5, random_type='normal'):
        self.hidden_units = hidden_units
        self.activation_function = activation_function
        self.regularization = regularization
        self.random_type = random_type
        self.bias = np.random.randn(hidden_units)  # Added
        self.beta_hat_d = None
        self.u_hat_d = None
        self.best_d = None


    def __input2hidden(self, X):
        activations = {
            'sigmoid': lambda x: 1 / (1 + np.exp(-(x + self.bias))),  # Added bias
            'relu': lambda x: (x + self.bias) * (x > 0),               # Added bias
            'sin': lambda x: np.sin(x + self.bias),                   # Added bias
            'tanh': lambda x: np.tanh(x + self.bias),                 # Added bias
            'leaky_relu': lambda x: np.maximum(0, x + self.bias) + 0.1 * np.minimum(0, x + self.bias)  # Add bias
        }
        return activations[self.activation_function](X)


    def __hidden2output(self, H):
        return np.dot(H, self.beta_hat_d)

    def fit(self, H, y, d=0):
        start_time = time.process_time()
        Ip = np.eye(H.shape[1]) * self.regularization
        HtH = np.dot(H.T, H) + Ip
        Hty = np.dot(H.T, y)
        self.beta_hat_d = np.linalg.solve(HtH + d * Ip, Hty)
        train_time = time.process_time() - start_time

        # Ensure beta_hat_d is reshaped to (n, 1)
        self.beta_hat_d = self.beta_hat_d.reshape(-1, 1)  # Reshape to (n, 1) if necessary
        return self.beta_hat_d, train_time

    def fit_random_effects(self, Z, y, Hb_hat):
        G_inv = np.linalg.inv(np.dot(Z.T, Z) + self.regularization * np.eye(Z.shape[1]))
        HTG_invH = np.dot(Hb_hat.T, np.dot(G_inv, Hb_hat))
        self.beta_hat = np.dot(np.linalg.inv(HTG_invH + self.regularization * np.eye(Hb_hat.shape[1])), np.dot(Hb_hat.T, np.dot(G_inv, y)))
        A = HTG_invH + self.regularization * np.eye(Hb_hat.shape[1])
        B_inv = np.linalg.inv(HTG_invH + d * np.eye(Hb_hat.shape[1]))
        self.beta_hat_d = np.dot(B_inv, np.dot(A, self.beta_hat))
        self.u_hat_d = np.dot(G_inv, np.dot(Z.T, (y - np.dot(Hb_hat, self.beta_hat_d))))

    def predict(self, H, Z):
        Hb_hat_d = np.dot(H, self.beta_hat_d)
        if self.u_hat_d is None:
            self.u_hat_d = np.zeros((Z.shape[1], 1))
        Zu_hat_d = np.dot(Z, self.u_hat_d)
        return Hb_hat_d + Zu_hat_d

    def score(self, H, Z, y):
        prediction = self.predict(H, Z)
        return np.sqrt(mean_squared_error(y, prediction))

    def count_accurate_predictions(self, H, Z, y, tolerance=0.1):
        predictions = self.predict(H, Z)
        accurate_predictions = np.abs(predictions - y) <= tolerance * y
        num_accurate = np.sum(accurate_predictions)
        return num_accurate, len(y), num_accurate / len(y)

    def beta_hat_d(self, H, y, d):
        Λ = np.dot(H.T, H)
        eigenvalues, eigenvectors = np.linalg.eigh(Λ)
        T = eigenvectors
        Λ = eigenvalues
        F_d = (Λ + d) / (Λ + 1)
        gamma_hat = np.dot(T.T, self.beta_hat_d)
        gamma_hat_d = F_d * gamma_hat
        return np.dot(T, gamma_hat_d)

    def beta_hat_k(self, H, y, d, k):
        if self.beta_hat_d is None:
            self.fit(H, y, d=d)

        beta_d = self.beta_hat_d
        if k < len(beta_d):
            return beta_d[k]
        else:
            raise ValueError(f"Index k={k} is out of bounds for beta_d with length {len(beta_d)}.")

def read_docx_table(file_path):
    doc = Document(file_path)
    data = []
    for table in doc.tables:
        for row in table.rows:
            row_data = [cell.text.strip() for cell in row.cells]
            data.append(row_data)
    return data

def calculate_d_mse(sigma_squared, lambda_values, gamma):
    numerator = np.sum(gamma ** 2) - sigma_squared
    denominator = np.sum(sigma_squared + lambda_values * gamma ** 2)
    return np.clip(numerator / denominator, 0, 1) if denominator != 0 else 0

def calculate_d_cl(sigma_squared, lambda_values, gamma):
    d_cl = 1 - sigma_squared * (np.sum(1 / (lambda_values + 1)) / np.sum(lambda_values * gamma ** 2))
    return np.clip(d_cl, 0, 1)

def calculate_d_gcv(RSS_d, n, tr_Hd):
    return RSS_d / (n - 1 - tr_Hd)

def d(H, beta_hat, sigma2):
    n = H.shape[0]
    p = H.shape[1]
    residuals = y - np.dot(H, beta_hat)
    residuals_T = residuals.T
    G_inv = np.linalg.inv(np.dot(H.T, H) + np.eye(H.shape[1]) * sigma2)
    sigma2 = np.dot(residuals_T, np.dot(G_inv, residuals)) / (n - p)
    eigenvalues, eigenvectors = np.linalg.eig(np.dot(H.T, H))
    alpha = np.dot(eigenvectors.T, beta_hat)
    numerator = np.sum([1 / (eigenvalue + 1) for eigenvalue in eigenvalues])
    denominator = np.sum([lambda_i * (alpha_i ** 2) / ((lambda_i + 1) ** 2) for lambda_i, alpha_i in zip(eigenvalues, alpha)])
    return 1 - (sigma2 * numerator / denominator)

def d1(H, beta_hat, sigma2):
    n = H.shape[0]
    p = H.shape[1]
    residuals = y - np.dot(H, beta_hat)
    residuals_T = residuals.T
    G_inv = np.linalg.inv(np.dot(H.T, H) + np.eye(H.shape[1]) * sigma2)
    sigma2 = np.dot(residuals_T, np.dot(G_inv, residuals)) / (n - p)
    eigenvalues, eigenvectors = np.linalg.eig(np.dot(H.T, H))
    alpha = np.dot(eigenvectors.T, beta_hat)
    numerator = np.sum([1 / (eigenvalue * (eigenvalue + 1)) for eigenvalue in eigenvalues])
    denominator = np.sum([(alpha[i] ** 2) / ((eigenvalues[i] + 1) ** 2) for i in range(len(eigenvalues))])
    return 1 - (sigma2 * numerator / denominator)

# Main code
file_path = 'Bostondata.docx'
data = read_docx_table(file_path)

df = pd.DataFrame(data[1:], columns=data[0])
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        pass

df = df.drop(columns=['TOWN', 'Nunmber', 'LLSTAT'])
column_names = ['CRIM', 'CHAS', 'AGE', 'RM2', 'NOX2', 'B1', 'LDIS', 'LMEDV']
df.columns = column_names

X = df.drop(columns=['LMEDV', 'NOX2']).values
Z = df['NOX2'].values.reshape(-1, 1)
y = df['LMEDV'].values.reshape(-1, 1)

scaler_X = MinMaxScaler()
scaler_Z = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
Z_scaled = scaler_Z.fit_transform(Z)

X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X_scaled, y, Z_scaled, test_size=0.2, random_state=42)

hidden_units = 700
input_dim = X_train.shape[1]
weights = np.random.randn(input_dim, hidden_units)

hidden_activations_train = np.dot(X_train, weights)
hidden_activations_test = np.dot(X_test, weights)

elm = MixedRegressionELM(hidden_units=hidden_units, activation_function='sigmoid')
hidden_activations_train = elm._MixedRegressionELM__input2hidden(hidden_activations_train)
hidden_activations_test = elm._MixedRegressionELM__input2hidden(hidden_activations_test)

beta_d, train_time = elm.fit(hidden_activations_train, y_train)
print(f"Train time: {train_time:.4f} seconds")

# Predictions
y_train_pred = elm.predict(hidden_activations_train, Z_train)
y_test_pred = elm.predict(hidden_activations_test, Z_test)

# Calculate RMSE for train and test
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

# Display RMSE results
print(f"RMSE: {train_rmse:.4f}")

# Separator
print("*" * 20)

# Final Beta values
print("Beta values:", beta_d.flatten())


In [ ]:
# Setting up values for k and d
k_values = [10**-5, 10**-4, 10**-3, 10**-2, 0.02, 0.1, 0.2, 1, 1.5, 2, 3, 4, 5]
d_values = [10**-5, 10**-4, 10**-3, 10**-2, 0.02, 0.1, 0.2, 1]

# Initialize storage for beta_hat_K_d and u_hat_K_d values for each combination
beta_hat_K_d_values = {}
u_hat_K_d_values = {}

# Print initial shapes for debugging
print("Initial shapes:")
print(f"Z_train shape: {Z_train.shape}")
print(f"hidden_activations_train shape: {hidden_activations_train.shape}")
print(f"y_train shape: {y_train.shape}")

for d in d_values:
    beta_d, train_time = elm.fit(hidden_activations_train, y_train, d=d)
    elm.beta_hat_d = beta_d

    for k in k_values:
        # Get dimensions
        n_samples = Z_train.shape[0]
        n_features_z = Z_train.shape[1]
        n_hidden = hidden_activations_train.shape[1]

        # Ensure y_train is 2D
        if len(y_train.shape) == 1:
            y_train_reshaped = y_train.reshape(-1, 1)
        else:
            y_train_reshaped = y_train

        # Calculate necessary matrices
        Z_T_Z = Z_train.T @ Z_train
        Z_T_y = Z_train.T @ y_train_reshaped
        Z_T_H = Z_train.T @ hidden_activations_train
        H_T_H = hidden_activations_train.T @ hidden_activations_train

        # Print shapes for debugging
        print("\nMatrix shapes after initial calculations:")
        print(f"Z_T_Z shape: {Z_T_Z.shape}")
        print(f"Z_T_y shape: {Z_T_y.shape}")
        print(f"Z_T_H shape: {Z_T_H.shape}")
        print(f"H_T_H shape: {H_T_H.shape}")

        # Calculate inverse matrices
        G_inv = np.linalg.inv(Z_T_Z + elm.regularization * np.eye(n_features_z))
        A = H_T_H + elm.regularization * np.eye(n_hidden)
        c_inv = np.linalg.inv(H_T_H + d * np.eye(n_hidden))
        B_inv = np.linalg.inv(H_T_H + k * np.eye(n_hidden))

        # Calculate beta_hat_K_d step by step with intermediate shape checks
        print("\nIntermediate calculation shapes:")

        # Modified calculation sequence
        temp1 = Z_train @ G_inv @ Z_train.T  # (404, 404)
        temp2 = hidden_activations_train.T @ (np.eye(n_samples) - temp1)  # (10, 404)
        temp3 = temp2 @ y_train_reshaped  # (10, 1)
        temp4 = c_inv @ temp3  # (10, 1)
        temp5 = A @ temp4  # (10, 1)
        beta_hat_K_d = B_inv @ temp5  # (10, 1)

        print(f"temp1 shape: {temp1.shape}")
        print(f"temp2 shape: {temp2.shape}")
        print(f"temp3 shape: {temp3.shape}")
        print(f"temp4 shape: {temp4.shape}")
        print(f"temp5 shape: {temp5.shape}")
        print(f"beta_hat_K_d shape: {beta_hat_K_d.shape}")

        beta_hat_K_d_values[(d, k)] = beta_hat_K_d

        # Calculate u_hat_K_d
        u_hat_K_d = G_inv @ (Z_T_y - Z_T_H @ beta_hat_K_d)
        print(f"u_hat_K_d shape: {u_hat_K_d.shape}")

        u_hat_K_d_values[(d, k)] = u_hat_K_d

        # Define prediction function
        def make_predict_function(beta_k_d, u_k_d):
            def predict(H, Z):
                return H @ beta_k_d + Z @ u_k_d
            return predict

        current_predict = make_predict_function(beta_hat_K_d, u_hat_K_d)

        # Calculate predictions
        y_train_pred = current_predict(hidden_activations_train, Z_train)
        y_test_pred = current_predict(hidden_activations_test, Z_test)

        # Calculate metrics
        train_rmse = np.sqrt(np.mean((y_train_reshaped - y_train_pred) ** 2))
        test_rmse = np.sqrt(np.mean((y_test.reshape(-1, 1) - y_test_pred) ** 2))

        # Print final results
        print("\nResults:")
        print(f"Parameters: d={d}, k={k}")
        print(f"Training RMSE: {train_rmse:.4f}")
        print(f"Test RMSE: {test_rmse:.4f}")
        print("*" * 70)

In [ ]:
y_all = np.concatenate((y_train, y_test), axis=0)
train_predictions = elm.predict(hidden_activations_train, Z_train)
test_predictions = elm.predict(hidden_activations_test, Z_test)

predictions_all = np.concatenate((train_predictions, test_predictions), axis=0)

plt.figure(figsize=(12, 6))
plt.scatter(np.arange(len(y_all)), y_all, color='blue', label='Actual LMEDV', alpha=0.6)
plt.plot(np.arange(len(predictions_all)), predictions_all, color='red', label='Model Predictions', linewidth=2)
plt.xlabel('Sample Index')
plt.ylabel('LMEDV')
plt.title('Actual vs. Predicted (Mixed Model)')
plt.legend()
plt.grid()
plt.show()



## STDV Train Predictions

In [ ]:
import statistics
print("Standard Deviation of train is % s "
                % (statistics.stdev(train_predictions.flatten())))

## STDV Test Predictions

In [ ]:
import statistics

# data - set
sample = (test_predictions)

# Prints standard deviation
# xbar is set to default value of 1
print("Standard Deviation of test is % s "
                % (statistics.stdev(sample.flatten())))

# Simple Regg

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.preprocessing import MinMaxScaler
from docx import Document

class SimpleLinearRegressionOld:
    def __init__(self):
        self.beta_hat = None
        self.beta_hat_d = None
        self.u_hat_d = None

    def fit(self, X, y, Z, regularization, d):
        n = Z.shape[0]
        G_inv = inv(Z @ Z.T + regularization * np.eye(n))
        XTG_inv_X = X.T @ G_inv @ X
        XTG_inv_X_inv = inv(XTG_inv_X)
        self.beta_hat = XTG_inv_X_inv @ X.T @ G_inv @ y

        A = (XTG_inv_X + d * np.eye(X.shape[1]))
        B_inv = inv(XTG_inv_X + np.eye(X.shape[1]))
        ###c_inv=inv(XTG_inv_X + k * np.eye(X.shape[1]))
        ###D=x @ G_inv @ y
        self.beta_hat_d = B_inv @ A @ self.beta_hat
        ###self.beta_hat_k_d = B_inv @ A @ c_inv @ D

        # Calculate u_hat_d
        self.u_hat_d = Z.T @ G_inv @ (y - X @ self.beta_hat)

        ### Calculate u_hat_K_d
        ###self.u_hat_K_d =  G_inv @ (Z.T @ y - Z.T @ X @ self.beta_hat_k_d)

    def predict(self, X, Z):
        Xbeta_hat_d = X @ self.beta_hat_d
        Zu_hat_d = Z @ self.u_hat_d
        return Xbeta_hat_d + Zu_hat_d

        ###def predict(self, X, Z):
        ###Xbeta_hat_K_d = X @ self.beta_hat_k_d
        ###Zu_hat_K_d = Z @ self.u_hat_K_d
        ###return Xbeta_hat_k_d + Zu_hat_K_d

    def score(self, X, y, Z):
        y_pred = self.predict(X, Z)
        return np.sqrt(np.mean((y - y_pred) ** 2))

# Function to read data from a Word document
def read_docx_table(file_path):
    doc = Document(file_path)
    data = []
    for table in doc.tables:
        for row in table.rows:
            data.append([cell.text.strip() for cell in row.cells])  # Strip to remove extra spaces
    return data

# Read data from Word file
file_path = 'Bostondata.docx'
data = read_docx_table(file_path)

# Create DataFrame
df = pd.DataFrame(data[1:], columns=data[0])
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        pass

# Drop unnecessary columns
df = df.drop(columns=['TOWN', 'Nunmber', 'LLSTAT'])
column_names = ['CRIM', 'CHAS', 'AGE', 'RM2', 'NOX2', 'B1', 'LDIS', 'LMEDV']
df.columns = column_names

# Split data into X, Z, and y variables
X = df.drop(columns=['LMEDV', 'NOX2']).values
Z = df['NOX2'].values.reshape(-1, 1)
y = df['LMEDV'].values.reshape(-1, 1)

# Scale data
scaler_X = MinMaxScaler()
scaler_Z = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
Z_scaled = scaler_Z.fit_transform(Z)

# Add noise to data
np.random.seed(42)
noise = np.random.normal(0, 0.5, size=X.shape)
X_noisy = X + noise

# Split data into training and test sets
split_index = int(0.8 * len(X_noisy))
X_train, X_test = X_noisy[:split_index], X_noisy[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Define ranges for k and d
k_values = [10**(-i) for i in range(5, 1)] + [0.02, 0.1, 0.2, 1, 1.5, 2, 5]
d_values = [10**(-i) for i in range(5, 1)] + [0.02, 0.1, 0.2, 1]

# Create model and train it for each combination of k and d
best_score = float('inf')
best_d = None
best_k = None
best_model = None

for d in d_values:
    for k in k_values:
        model = SimpleLinearRegressionOld()
        regularization = k  # Use k as regularization parameter
        model.fit(X_train, y_train, Z_scaled[:split_index], regularization, d)

        # Calculate the score
        train_score = model.score(X_train, y_train, Z_scaled[:split_index])
        test_score = model.score(X_test, y_test, Z_scaled[split_index:])

        # Print results for the current combination of d and k
        print(f"Testing d={d:.5f}, k={k:.5f} => Train RMSE: {train_score:.4f}, Test RMSE: {test_score:.4f}")

        # Save the best model based on test score
        if test_score < best_score:
            best_score = test_score
            best_d = d
            best_k = k
            best_model = model

# Print the best results
print("-" * 40)
print(f"Best Test RMSE: {best_score:.4f} for d={best_d:.5f}, k={best_k:.5f}")
print("-" * 40)

# Plot comparison of predictions and actual values using the best model
train_predictions = best_model.predict(X_train, Z_scaled[:split_index])
test_predictions = best_model.predict(X_test, Z_scaled[split_index:])

y_all = np.concatenate((y_train, y_test), axis=0)
predictions_all = np.concatenate((train_predictions, test_predictions), axis=0)

plt.figure(figsize=(12, 6))
plt.scatter(np.arange(len(y_all)), y_all, color='blue', label='Actual LMEDV')
plt.plot(np.arange(len(predictions_all)), predictions_all, color='red', label='Model Predictions', linewidth=2)
plt.xlabel('Sample Index')
plt.ylabel('LMEDV')
plt.title('Actual vs. Predicted (Simple Linear Regression)')
plt.legend()
plt.grid()
plt.show()
